 Проект 4. Авиарейсы без потерь 

Задание 4.1 

База данных содержит список аэропортов практически всех крупных городов России. В большинстве городов есть только один аэропорт. Исключение составляет:

In [ ]:
SELECT city
FROM dst_project.airports
GROUP BY city
HAVING count(city)>1

# Moscow, Ulyanovsk

Задание 4.2 

Вопрос 1. Таблица рейсов содержит всю информацию о прошлых, текущих и запланированных рейсах. Сколько всего статусов для рейсов определено в таблице?

In [ ]:
SELECT count(DISTINCT status)
FROM dst_project.flights

# 6

Вопрос 2. Какое количество самолетов находятся в воздухе на момент среза в базе (статус рейса «самолёт уже вылетел и находится в воздухе»). 

In [ ]:
SELECT count(aircraft_code)
FROM dst_project.flights
WHERE status ='Departed'

# 58

Вопрос 3. Места определяют схему салона каждой модели. Сколько мест имеет самолет модели (Boeing 777-300)? 

In [ ]:
SELECT count(seat_no)
FROM dst_project.seats
WHERE aircraft_code = '773'

# 402

Вопрос 4. Сколько состоявшихся (фактических) рейсов было совершено между 1 апреля 2017 года и 1 сентября 2017 года?

Здесь и далее состоявшийся рейс означает, что он не отменён, и самолёт прибыл в пункт назначения.

In [ ]:
SELECT count(*)
FROM dst_project.flights
WHERE actual_arrival > to_date('2017-04-01', 'YYYY-MM-DD')
    and actual_arrival < to_date('2017-09-01', 'YYYY-MM-DD')
    and status != 'Cancelled' and status = 'Arrived'

# 74227

 Задание 4.3

Вопрос 1. Сколько всего рейсов было отменено по данным базы?

In [ ]:
SELECT count(*)
FROM dst_project.flights
WHERE status = 'Cancelled'

# 437

Вопрос 2. Сколько самолетов моделей типа Boeing, Sukhoi Superjet, Airbus находится в базе авиаперевозок? 

In [ ]:
WITH p AS
  (SELECT 'Airbus' AS p_name,
          *
   FROM dst_project.aircrafts
   WHERE model like ('Airbus%')
   UNION SELECT 'Boeing' AS p_name,
                *
   FROM dst_project.aircrafts
   WHERE model like ('Boeing%')
   UNION SELECT 'Sukhoi Superjet' AS p_name,
                *
   FROM dst_project.aircrafts
   WHERE model like ('Sukhoi Superjet%') )
SELECT p_name,
       count(*)
FROM p
GROUP BY p_name

# Boeing: 3, Sukhoi Superjet: 1, Airbus: 3

Вопрос 3. В какой части (частях) света находится больше аэропортов? 

In [ ]:
SELECT substring(timezone
                 FROM 0
                 FOR position('/' in timezone)) AS part_of_world,
       count(airport_code)
FROM dst_project.airports
GROUP BY 1

# Europe, Asia

Вопрос 4. У какого рейса была самая большая задержка прибытия за все время сбора данных? Введите id рейса (flight_id). 

In [ ]:
SELECT flight_id,
       max(actual_arrival - scheduled_arrival) AS delay
FROM dst_project.flights
GROUP BY flight_id
ORDER BY delay DESC NULLS LAST
LIMIT 1

# 157571

Задание 4.4 

Вопрос 1. Когда был запланирован самый первый вылет, сохраненный в базе данных? 

In [ ]:
SELECT min(scheduled_departure)
FROM dst_project.flights

# 14.08.2016

Вопрос 2. Сколько минут составляет запланированное время полета в самом длительном рейсе? 

In [ ]:
SELECT max(extract(epoch
                   FROM (scheduled_arrival - scheduled_departure))) / 60
FROM dst_project.flights

# 530

Вопрос 3. Между какими аэропортами пролегает самый длительный по времени запланированный рейс? 

In [ ]:
SELECT departure_airport,
       arrival_airport,
       status,
       max(scheduled_arrival - scheduled_departure) AS total_time
FROM dst_project.flights
WHERE status = 'Scheduled'
  AND departure_airport in ('DME',
                            'SVO')
  AND arrival_airport in ('UUS',
                          'AAQ',
                          'PCS')
GROUP BY departure_airport,
         arrival_airport,
         status
ORDER BY total_time DESC
LIMIT 1

# DME - UUS

Вопрос 4. Сколько составляет средняя дальность полета среди всех самолетов в минутах? Секунды округляются в меньшую сторону (отбрасываются до минут). 

In [ ]:
SELECT floor(avg(extract(epoch
                         FROM (actual_arrival - actual_departure))) / 60)
FROM dst_project.flights

# 128

 Задание 4.5

Вопрос 1. Мест какого класса у SU9 больше всего? 

In [ ]:
SELECT fare_conditions,
       count(*) total
FROM dst_project.seats
WHERE aircraft_code = 'SU9'
GROUP BY fare_conditions
ORDER BY total DESC
LIMIT 1

# Economy 

Вопрос 2. Какую самую минимальную стоимость составило бронирование за всю историю? 

In [ ]:
SELECT min(total_amount)
FROM dst_project.bookings

# 3400

Вопрос 3. Какой номер места был у пассажира с id = 4313 788533? 

In [ ]:
SELECT seat_no
FROM dst_project.tickets AS t
JOIN dst_project.boarding_passes bp ON t.ticket_no = bp.ticket_no
WHERE passenger_id = '4313 788533'

# 2A

 Задание 5.1

Вопрос 1. Анапа — курортный город на юге России. Сколько рейсов прибыло в Анапу за 2017 год?

In [ ]:
SELECT count(*)
FROM dst_project.airports a
JOIN dst_project.flights f ON a.airport_code = f.arrival_airport
WHERE a.city = 'Anapa'
  AND date_part('year', actual_arrival) = 2017

# 486

Вопрос 2. Сколько рейсов из Анапы вылетело зимой 2017 года?

In [ ]:
SELECT count(*)
FROM dst_project.airports a
JOIN dst_project.flights f ON a.airport_code = f.departure_airport
WHERE a.city = 'Anapa'
  AND date_part('year', actual_departure) = 2017
  AND date_part('month', actual_departure) in (1,
                                               2,
                                               12)
  
# 127

Вопрос 3. Посчитайте количество отмененных рейсов из Анапы за все время.

In [ ]:
SELECT count(*)
FROM dst_project.airports a
JOIN dst_project.flights f ON a.airport_code = f.departure_airport
WHERE a.city = 'Anapa'
  AND f.status = 'Cancelled'

# 1

Вопрос 4. Сколько рейсов из Анапы не летают в Москву?

In [ ]:
SELECT count(flight_id)
FROM dst_project.flights f
JOIN dst_project.airports a ON a.airport_code = f.departure_airport
JOIN dst_project.airports ap ON ap.airport_code = f.arrival_airport
WHERE a.city = 'Anapa'
  AND ap.city != 'Moscow'

# 453

Вопрос 5. Какая модель самолета летящего на рейсах из Анапы имеет больше всего мест?

In [ ]:
WITH flight_seets AS
  (SELECT f.flight_id,
          ac.model,
          count(*) total
   FROM dst_project.flights f
   JOIN dst_project.airports ap ON ap.airport_code = f.departure_airport
   JOIN dst_project.aircrafts ac ON ac.aircraft_code = f.aircraft_code
   JOIN dst_project.seats s ON s.aircraft_code = f.aircraft_code
   WHERE ap.city = 'Anapa'
   GROUP BY f.flight_id,
            ac.model)
SELECT DISTINCT model,
                total
FROM flight_seets
ORDER BY total DESC
LIMIT 1

# Boeing 737-300